In [1]:
import pandas as pd
import numpy as np
import pdb#将dbug模块装载，方便调试
#读取文件夹中parents内容
parents=pd.read_excel("train_gene.xlsx",keep_default_na=False)
#读取文件夹中map内容
MAP=pd.read_excel("Map_clean.xlsx",keep_default_na=False)

In [2]:
def function2(input_figuers):#将一行数据拆分并存储进一个全新的列表中
    #split_data = input_figuers.str.split("", expand=True)
    split_data = input_figuers.apply(lambda x: x.split("/") if '/' in x else [x])
# 将拆分后的字符存储到一个新的数组
    result_array = split_data.values.flatten()
#删除nontype
    result_array = [item for item in result_array if item is not None]

# 打印结果
    return result_array

In [3]:
def merge_nested_lists(nested_list):
    merged_list = [item for sublist in nested_list for item in sublist]
    return merged_list

In [4]:
def replace_dash(lst):#将-，-合成为-/-
    # 遍历列表
    i = 0
    while i < len(lst) - 1:
        # 如果当前元素和下一个元素都是 '-'
        if lst[i] == '-' and lst[i + 1] == '-':
            # 将当前元素替换为 '-/-'
            lst[i] = '-/-'
            # 移除下一个元素
            lst.pop(i + 1)
        else:
            # 否则，继续遍历下一个元素
            i += 1
    return lst

In [5]:
def check_list(lst):
    # 遍历列表
    for item in lst:
        # 如果元素为 'NA' 或 '-/-'，则返回 False
        if item == 'NA' or item == '-/-':
            return False
    # 如果列表中没有 'NA' 或 '-/-'，则返回 True
    return True

In [6]:
def process_elements(element_list):#将处理后的列表恢复成原来格式
    processed_elements = [element_list[0]]  # Include the first element
    i = 1
    while i < len(element_list):
        current_element = element_list[i]
        if '/' in current_element:
            processed_elements.append(current_element)
        elif current_element == 'A' or current_element == 'T' or current_element == 'C' or current_element == 'G':
            next_index = i + 1
            if next_index < len(element_list) and '/' not in element_list[next_index]:
                processed_elements.append(current_element + '/' + element_list[next_index])
                i += 1  # Move to the next element
            else:
                processed_elements.append(current_element)
        i += 1
    return processed_elements

In [7]:
def function1(input_figures):#缺失率和MAF函数
    A=0
    T=0
    C=0
    G=0
    missing=0
    for i in range(1,len(input_figures)):
        if(input_figures[i]=='A'):
            A=A+1
        elif(input_figures[i]=='T'):
            T=T+1
        elif(input_figures[i]=='C'):
            C=C+1
        elif(input_figures[i]=='G'):
            G=G+1
        elif( input_figures[i]=='-'):
            missing=missing+1
        elif(input_figures[i]=='NA'):
            missing=missing+2
    total_sum=A+T+C+G+missing#计算缺失率，这是总数
    if (total_sum==0):
        return False
    else:
        missing_rate=missing/total_sum#这是缺失率
        a=A/total_sum
        t=T/total_sum
        c=C/total_sum
        g=G/total_sum
        my_list=[a,t,c,g]
        # 将零值替换为正无穷
        replaced_list = [x if x != 0 else float('inf') for x in my_list]
    
    # 找到替换后列表中的最小值
        MAF= min(replaced_list)#这是该行的MAF
        #找到最大值
        data={'A/A':a, 'T/T': t, 'C/C': c, 'G/G': g}
        MAXium_identifier=max(data,key=data.get)
        if (0<=missing_rate<=0.4 and MAF>=0.05):
            return str(MAXium_identifier)#保留着行数据，并返回缺失值的str进行修改
        else:
            return False #若返回为403则删除

In [8]:
def function3(input_figures):#用于对一行的NA和-/-值进行填补,找到最高含量的碱基
    T=0
    C=0
    G=0
    A=0
    for i in range(1,len(input_figures)):
        if(input_figures[i]=='A'):
            A=A+1
        elif(input_figures[i]=='T'):
            T=T+1
        elif(input_figures[i]=='C'):
            C=C+1
        elif(input_figures[i]=='G'):
            G=G+1
    a=A/20
    t=T/20
    c=C/20
    g=G/20
    #找到最大值
    data={'A/A':a, 'T/T': t, 'C/C': c, 'G/G': g}
    MAXium_identifier=max(data,key=data.get)
    for i in range(1, len(input_figures)):  # 从索引1开始遍历，跳过第一个元素 '9_12663640'
        if input_figures[i] == 'NA' or input_figures[i] == '-/-':  # 修正条件判断
            input_figures[i] = MAXium_identifier
    return input_figures
    

In [9]:
N=[]
for i in range(0,len(parents)):#删掉多余不符合要求的行
    if (function1(merge_nested_lists(function2(parents.iloc[i])))==False):
        value=parents.iloc[i, 0]
        N.append(value)
        #parents = parents.drop(index=i)
df = parents[~parents['id'].isin(N)]

In [10]:
for i in range(0,len(df)):
    if (check_list(replace_dash(merge_nested_lists(function2(df.iloc[i,1:501]))))==False):
        df.iloc[i]=process_elements(function3(replace_dash(merge_nested_lists(function2(df.iloc[i].copy())))))
        #df.loc[i]=process_elements(function3(replace_dash(merge_nested_lists(function2(df.loc[i].copy())))))

C:\Users\杜溥泉\AppData\Local\Temp\ipykernel_13168\3989131527.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.iloc[i]=process_elements(function3(replace_dash(merge_nested_lists(function2(df.iloc[i].copy())))))


In [11]:
new_parents= df.replace(False, np.nan, inplace=False).dropna(inplace=False)

In [12]:
# 将Map的DNApolymorphismid列转换为一个字典，键为DNApolymorphismid，值为索引  
order_list = MAP['DNApolymorphismid'].to_list()
new_df=[]
# 使用这个字典来对parents进行排序  
'''for i in order_list:
    for row in parents.itertuples(index=True, name=None): 
        print(row)'''

# 创建一个CategoricalDtype，使用order_list作为类别顺序  
cat_type = pd.CategoricalDtype(categories=order_list, ordered=True)  
  
# 将parents DataFrame的id列转换为分类数据类型  
df['id'] = df['id'].astype(cat_type)  
  
# 根据id列的分类顺序对DataFrame进行排序  
parents_sorted = df.sort_values(by='id')  
  
# 查看排序后的DataFrame  
print(parents_sorted)

C:\Users\杜溥泉\AppData\Local\Temp\ipykernel_13168\325328206.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['id'] = df['id'].astype(cat_type)


               id EMF2100288777 EMF2200591395 EMF2100289528 EMF2200590276  \
776    9_12652123           G/G           A/A           A/A           G/G   
712    9_12652277           A/A           A/A           A/A           C/A   
762    9_12652152           G/A           G/A           A/A           G/A   
15565  9_12667060           A/A           A/G           A/G           A/A   
15573  9_12667073           G/G           A/G           G/G           G/G   
...           ...           ...           ...           ...           ...   
15602         NaN           G/G           G/G           G/G           G/G   
15686         NaN           G/G           G/G           G/G           G/A   
15690         NaN           G/G           G/G           G/G           G/C   
15714         NaN           C/C           C/C           C/C           C/C   
15722         NaN           C/C           C/C           C/A           A/A   

      EMF2100769652 EMF1900599186 EMF2201896517 EMF2100775708 EMF1700596199

In [13]:
parents_sorted.to_excel('traingene_clean.xlsx',index=False)


In [14]:
print(N)

['9_12654442', '9_12654436', '9_12612178', '9_12612167', '9_12612161', '9_12654402', '9_12612157', '9_12612156', '9_12612134', '9_12654334', '9_12612131', '9_12612129', '9_12612127', '9_12654084', '9_12612122', '9_12612117', '9_12654003', '9_12612115', '9_12612114', '9_12653997', '9_12612104', '9_12612102', '9_12612080', '9_12612066', '9_12615843', '9_12615841', '9_12615945', '9_12616957', '9_12616951', '9_12653658', '9_12653628', '9_12616937', '9_12616936', '9_12653595', '9_12616930', '9_12653579', '9_12616917', '9_12653557', '9_12616894', '9_12653472', '9_12616824', '9_12616803', '9_12616779', '9_12616739', '9_12616724', '9_12616703', '9_12653296', '9_12616691', '9_12616678', '9_12616676', '9_12653215', '9_12616665', '9_12616664', '9_12616642', '9_12614207', '9_12653136', '9_12653114', '9_12653092', '9_12614083', '9_12614082', '9_12614081', '9_12614078', '9_12614077', '9_12653012', '9_12614032', '9_12614007', '9_12613999', '9_12613927', '9_12613886', '9_12613880', '9_12613876', '9_12